**Programador:** Marco Gutierrez. Si se tiene alguna consulta, mandarla al siguiente [email](mailto:mgutierrezc@up.edu.pe)

# Obteniendo la información para el Scrapping

Importamos las librerias para el scrapping

In [76]:
import pandas as pd
import string
import googlemaps
import numpy as np
import os
import re

## Importando las provincias/distritos a analizar

Primero, leeremos un archivo que contiene los elementos de los ubigeos del INEI. Se puede descargar entrando [aquí](http://webinei.inei.gob.pe:8080/sisconcode/proyecto/index.htm?proyectoTitulo=UBIGEO&proyectoId=3) y dando click a la opción **Excel** en la pestaña *Busqueda por Ubicación Geográfica*

In [77]:
ubigeos = pd.read_excel('ubigeo_inei.xls',skiprows=1)
ubigeos.head()

,Unnamed: 0,DEPARTAMENTO,Unnamed: 2,Unnamed: 3,PROVINCIA,DISTRITO
0,NaN,01 Amazonas,NaN,NaN,,
1,NaN,01 Amazonas,NaN,NaN,01 Chachapoyas,
2,NaN,01 Amazonas,NaN,NaN,01 Chachapoyas,01 Chachapoyas
3,NaN,01 Amazonas,NaN,NaN,01 Chachapoyas,02 Asunción
4,NaN,01 Amazonas,NaN,NaN,01 Chachapoyas,03 Balsas


Necesitamos las provincias y distritos de esta base, así que limpiaremos los valores de estas variables

In [78]:
ubigeos["PROVINCIA"] = ubigeos["PROVINCIA"].str.replace('^\d* ', "", flags=re.I, regex=True)
ubigeos["DISTRITO"] = ubigeos["DISTRITO"].str.replace('^\d* ', "", flags=re.I, regex=True)
ubigeos.head()

,Unnamed: 0,DEPARTAMENTO,Unnamed: 2,Unnamed: 3,PROVINCIA,DISTRITO
0,NaN,01 Amazonas,NaN,NaN,,
1,NaN,01 Amazonas,NaN,NaN,Chachapoyas,
2,NaN,01 Amazonas,NaN,NaN,Chachapoyas,Chachapoyas
3,NaN,01 Amazonas,NaN,NaN,Chachapoyas,Asunción
4,NaN,01 Amazonas,NaN,NaN,Chachapoyas,Balsas


Necesitaremos estas dos columnas como listas, pues después las usaremos para crear un DataFrame donde guardaremos los resultados del scrapping

In [79]:
# Creando las listas
provincias = list(ubigeos['PROVINCIA'])
distritos = list(ubigeos['DISTRITO'])

Limpiando las listas

In [80]:
# Eliminando duplicados
provincias = set(provincias)
distritos = set(distritos)

# Eliminando algunos elementos inutiles ('', nan)
provincias = list(filter(lambda x: str(x) != 'nan', provincias))
distritos = list(filter(lambda x: str(x) != 'nan', distritos))

provincias.remove('')
provincias.remove('PROVINCIA')
distritos.remove('')

Eliminando las tildes

In [81]:
# Definimos una función para eliminar las tildes
def normalize(s):
    replacements = (
        ("á", "a"),
        ("à", "a"),
        ("é", "e"),
        ("è", "e"),
        ("í", "i"),
        ("ì", "i"),
        ("ó", "o"),
        ("ò", "o"),
        ("ú", "u"),
        ("ù", "u")
    )
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    return s

In [82]:
# La aplicamos a cada elemento de nuestras provincias/distritos
nro_provs = len(provincias)
for i in range(nro_provs):
    provincias[i] = normalize(provincias[i])
    # De paso, les quitaremos espacios
    provincias[i] = provincias[i].strip()

nro_distrs = len(distritos)
for i in range(nro_distrs):
    distritos[i] = normalize(distritos[i])
    distritos[i] = distritos[i].strip()

In [83]:
print(provincias)
# print(distritos)

['Chepen', 'Huarochiri', 'Espinar', 'Castrovirreyna', 'Moyobamba', 'Caraveli', 'Palpa', 'Ocros', 'Manu', 'San Martin', 'Chachapoyas', 'Candarave', 'Paita', 'Cangallo', 'Angaraes', 'El Collao', 'Leoncio Prado', 'Tarma', 'Jaen', 'Pachitea', 'Cotabambas', 'Contralmirante Villar', 'San Ignacio', 'Condesuyos', 'Antabamba', 'Huallaga', 'Tacna', 'Tumbes', 'Chota', 'Huaura', 'Puerto Inca', 'Castilla', 'Huancane', 'Trujillo', 'Canas', 'Zarumilla', 'Pataz', 'Coronel Portillo', 'Pisco', 'Bongara', 'Yauyos', 'Ilo', 'Tarata', 'Jorge Basadre', 'Santa', 'Huacaybamba', 'Churcampa', 'Tayacaja', 'Moho', 'Cutervo', 'Cusco', 'Asuncion', 'Julcan', 'Huancabamba', 'Dos de Mayo', 'Huaraz', 'Sihuas', 'Aija', 'Cajamarca', 'Pacasmayo', 'Mariscal Ramon Castilla', 'Satipo', 'Cañete', 'Maynas', 'Ucayali', 'San Roman', 'Carlos Fermin Fitzcarrald', 'Tahuamanu', 'Contumaza', 'Lambayeque', 'Gran Chimu', 'Paucartambo', 'Bellavista', 'San Miguel', 'Islay', 'Cajabamba', 'Junin', 'Arequipa', 'Ambo', 'Purus', 'Aymaraes', 'A

# Scrapping de Distancias y Tiempos de viaje

Para realizar este scrapping, se tomará como referencia esta [guía](https://www.datahubbs.com/google-maps-python/)

## Matrices base a nivel de Provincias

Lo primero será crear un DataFrame con los distritos/provincias de origen y de llegada.

In [84]:
first_col = ["Origin Province\Destination Province"]
prov_distances = pd.DataFrame({"Origin Province\Destination Province": provincias}, 
                        columns = first_col+provincias)
prov_times = pd.DataFrame({"Origin Province\Destination Province": provincias}, 
                        columns = first_col+provincias)
prov_distances

,Origin Province\Destination Province,Chepen,Huarochiri,Espinar,Castrovirreyna,Moyobamba,Caraveli,Palpa,Ocros,Manu,...,Caylloma,Tocache,Puno,La Mar,Huamalies,Putumayo,Urubamba,Sullana,Quispicanchi,Otuzco
0,Chepen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Huarochiri,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Espinar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Castrovirreyna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Moyobamba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,Putumayo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192,Urubamba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193,Sullana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194,Quispicanchi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Creando listas con los origenes/destinos para reemplazar en el dataframe

In [129]:
# Como queremos iterar sobre los nombres de las provincias
origen_destino_prov=[]

for prov_row in provincias:
    origen_destino_prov.append(prov_row+"+"+"Peru")

origen_destino_prov

['Chepen+Peru',
 'Huarochiri+Peru',
 'Espinar+Peru',
 'Castrovirreyna+Peru',
 'Moyobamba+Peru',
 'Caraveli+Peru',
 'Palpa+Peru',
 'Ocros+Peru',
 'Manu+Peru',
 'San Martin+Peru',
 'Chachapoyas+Peru',
 'Candarave+Peru',
 'Paita+Peru',
 'Cangallo+Peru',
 'Angaraes+Peru',
 'El Collao+Peru',
 'Leoncio Prado+Peru',
 'Tarma+Peru',
 'Jaen+Peru',
 'Pachitea+Peru',
 'Cotabambas+Peru',
 'Contralmirante Villar+Peru',
 'San Ignacio+Peru',
 'Condesuyos+Peru',
 'Antabamba+Peru',
 'Huallaga+Peru',
 'Tacna+Peru',
 'Tumbes+Peru',
 'Chota+Peru',
 'Huaura+Peru',
 'Puerto Inca+Peru',
 'Castilla+Peru',
 'Huancane+Peru',
 'Trujillo+Peru',
 'Canas+Peru',
 'Zarumilla+Peru',
 'Pataz+Peru',
 'Coronel Portillo+Peru',
 'Pisco+Peru',
 'Bongara+Peru',
 'Yauyos+Peru',
 'Ilo+Peru',
 'Tarata+Peru',
 'Jorge Basadre+Peru',
 'Santa+Peru',
 'Huacaybamba+Peru',
 'Churcampa+Peru',
 'Tayacaja+Peru',
 'Moho+Peru',
 'Cutervo+Peru',
 'Cusco+Peru',
 'Asuncion+Peru',
 'Julcan+Peru',
 'Huancabamba+Peru',
 'Dos de Mayo+Peru',
 'Huar

## TODO: Matrices base a nivel de distritos

Crearemos nuestra matriz de la siguiente manera

In [127]:
first_col = ["Origin Province\Destination Province"]
dist_distances = pd.DataFrame({"Origin Province\Destination Province": distritos}, 
                        columns = first_col+distritos)
dist_times = pd.DataFrame({"Origin Province\Destination Province": distritos}, 
                        columns = first_col+distritos)
dist_distances

,Origin Province\Destination Province,Pariahuanca,Sondorillo,Tocache,Tintay Puncu,Curimana,Chazuta,Candarave,Marcara,Coyllurqui,...,Cochorco,Acos Vinchos,Colcamar,Pillpinto,Oropesa,Yauyos,La Banda de Shilcayo,Chirinos,Putumayo,Velille
0,Pariahuanca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sondorillo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Tocache,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Tintay Puncu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Curimana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1718,Yauyos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1719,La Banda de Shilcayo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1720,Chirinos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1721,Putumayo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Creando listas con los origenes/destinos para reemplazar en el dataframe

In [131]:
# Como queremos iterar sobre los nombres de las provincias
origen_destino_dist=[]

for dist_row in distritos:
    origen_destino_dist.append(dist_row+"+"+"Peru")

origen_destino_dist

['Pariahuanca+Peru',
 'Sondorillo+Peru',
 'Tocache+Peru',
 'Tintay Puncu+Peru',
 'Curimana+Peru',
 'Chazuta+Peru',
 'Candarave+Peru',
 'Marcara+Peru',
 'Coyllurqui+Peru',
 'Tisco+Peru',
 'Chaca+Peru',
 'Jaen+Peru',
 'Carabamba+Peru',
 'Pampamarca+Peru',
 'Shapaja+Peru',
 'Sonche+Peru',
 'Carmen de la Legua Reynoso+Peru',
 'La Matanza+Peru',
 'Urpay+Peru',
 'Vinchos+Peru',
 'Pacora+Peru',
 'Paranday+Peru',
 'Valera+Peru',
 'Paucarbamba+Peru',
 'Cosme+Peru',
 'Achaya+Peru',
 'Santiago de Surco+Peru',
 'Santa Eulalia+Peru',
 'Magdalena del Mar+Peru',
 'Huaynacotas+Peru',
 'Caujul+Peru',
 'Mollepata+Peru',
 'Santa Catalina de Mossa+Peru',
 'Quinuabamba+Peru',
 'Callalli+Peru',
 'Salaverry+Peru',
 'Huarango+Peru',
 'Moho+Peru',
 'Breña+Peru',
 'Sicuani+Peru',
 'Nanchoc+Peru',
 'Magdalena de Cao+Peru',
 'Huac-Huas+Peru',
 'San Pedro de Casta+Peru',
 'Tarapoto+Peru',
 'Coayllo+Peru',
 'Lloque+Peru',
 'Anchihuay+Peru',
 'Pisac+Peru',
 'Cahuapanas+Peru',
 'Marmot+Peru',
 'Omas+Peru',
 'Copani+P

## ¿Cómo usar el paquete `googlemaps`?

Ahora que ya tenemos todos los datos listos, nos falta activar la clase `googlemaps` para que podamos extraer tiempos y distancias. Ello se puede hacer así:

1. Importamos la librería googlemaps (Hecho al inicio de este jupyter)
1. Activamos el API `Distance Matrix API` en Google Cloud Platform (se puede encontrar una guía [aquí](https://www.youtube.com/watch?v=n1UorU1PALk))
1. Creamos una API key para poder usar el item anterior (guía [aquí](https://www.youtube.com/watch?v=2_HZObVbe-g))
1. Activaremos el cobro por el uso del API (Nos daran una prueba gratuita. La guía está [aquí](https://support.google.com/googleapi/answer/6158867?hl=en))

Sin un API key no podrán seguir el proceso, pero es de uso **personal**. Por ello, nunca la debemos escribir directamente en nuestro código; sin embargo, aprenderemos como utilizarla manteniendola privada y lo haremos usando **variables del sistema**

En el Menú de Inicio de Windows, busquen "environment" o "variables". Escojan la opción **"Editar las variables de entorno del sistema"**
![](images\env1.jpg)

Abranlo y entren a la pestaña `Opciones Avanzadas`

![](images\env2.jpg)

Hagan click a la opción `Variables de entornno`
![](images\env3.jpg)

Vayan a la sección `Variables de entorno` y denle click a **Nueva...**
![](images\env4.jpg)

Coloquen el nombre que prefieran (en mi caso, usé  `maps_key`) y, como valor, la API Key correspondiente (no la deben colocar entre comillas)
![](images\env5.jpg)

Denle Aceptar a esta ventana y luego a las anteriores hasta que se cierren todas. Cuando estén cerradas, reinicien su computadora y ya deberían poder usarla

## Usando `googlemaps` para crear una función que calcule distancias/tiempos

Cuando ya tengan su API Key lista, llamenla de la siguiente manera

In [86]:
key = os.environ.get('maps_key')

Ahora, activaremos la clase google maps usando nuestro key

In [87]:
gmaps = googlemaps.Client(key)

In [141]:
def matrix_blocks(first_row, first_column, final_row_plus_one, final_col, df_distances, df_times, origin_dest):
    """
    Función para completar la matriz por bloques que comiencen desde "first_row" y "first_column" hasta "iterations_row" 
    y "iterations_column". Se creó porque ejecutar el sgte codigo para todas las celdas toma mucho tiempo en ejecutarse.
    
    Para hacer más eficiente la tarea, se puede ejecutar esto por partes hasta completarlo. Además, permite convertir la tarea
    en una grupal. Un equipo puede hacer que cada uno de sus miembros complete ciertos bloques (e.g. linea 1 hasta 12, otro 
    linea 12 hasta 24, y así sucesivamente)
    """
    # Codigo base para cada fila (comenzando con la 0)
    for row in range(first_row, final_row_plus_one):
        print(row)
        for col in range(first_column, final_col):
            # Definiendo un índice para nuestra lista con los origenes/destinos en función a la columna del dataframe
            # en la que queremos el resultado
            index_destination = col-1

            # Si las filas y columnas son consecutivas, las distancias y tiempo son de un sitio al mismo sitio, asi que 
            # debemos controlar ese factir
            if col-1==row:
                df_distances.iloc[row,col] = '0 km'
                df_times.iloc[row,col] = '0 mins'

            else:
                # Obteniendo las distancias/tiempos correspondientes: 
                dist_time_mat = gmaps.distance_matrix(origins=origin_dest[row], destinations=origin_dest[index_destination], mode='driving')
                distance_duration = dist_time_mat['rows'][0]['elements'][0]

                # No siempre es posible viajar de un lugar a otro por restricciones geográficas, así que debemos definir qué ocurre
                # en casos así
                if distance_duration.get('distance'):
                    df_distances.iloc[row,col] = distance_duration['distance']['text']
                else:
                    df_distances.iloc[row,col] = '-'

                if distance_duration.get('duration'):
                    df_times.iloc[row,col] = distance_duration['duration']['text']
                else:
                    df_times.iloc[row,col] = '-'            

                # Para debuggear:
                # print('Origen: '+origen_destino[row]+' | Destino: '+origen_destino[index_destination]) 
                # print(prov_distances.iloc[row,col])

### Aplicando la función a las Matrices para Provincias

In [148]:
primera_fila = 71
primera_col = 1
ult_fila_mas_uno = len(provincias)
ult_col = len(provincias)+1

matrix_blocks(primera_fila, primera_col, ult_fila_mas_uno, ult_col, prov_distances, prov_times, origen_destino_prov)

71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195


In [149]:
prov_distances

,Origin Province\Destination Province,Chepen,Huarochiri,Espinar,Castrovirreyna,Moyobamba,Caraveli,Palpa,Ocros,Manu,...,Caylloma,Tocache,Puno,La Mar,Huamalies,Putumayo,Urubamba,Sullana,Quispicanchi,Otuzco
0,Chepen,0 km,894 km,"1,776 km","1,196 km",673 km,"1,474 km","1,089 km",592 km,-,...,"1,698 km",673 km,"1,985 km",707 km,634 km,-,"1,801 km",325 km,"1,840 km",218 km
1,Huarochiri,896 km,0 km,"1,096 km",516 km,"1,294 km",794 km,409 km,487 km,-,...,"1,018 km",869 km,"1,305 km",189 km,674 km,-,"1,121 km","1,219 km","1,160 km",828 km
2,Espinar,"1,992 km","1,309 km",0 km,874 km,"2,068 km",663 km,895 km,"1,583 km",-,...,78.5 km,"1,643 km",252 km,"1,284 km","1,769 km",-,284 km,"2,315 km",198 km,"1,924 km"
3,Castrovirreyna,"1,083 km",400 km,"1,007 km",0 km,"1,336 km",705 km,320 km,674 km,-,...,929 km,911 km,"1,216 km",376 km,860 km,-,844 km,"1,406 km",883 km,"1,015 km"
4,Moyobamba,673 km,"1,291 km","2,173 km","1,335 km",0 km,"1,871 km","1,487 km","1,015 km",-,...,"2,095 km",425 km,"2,382 km","1,104 km",850 km,-,"1,916 km",716 km,"1,955 km",889 km
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,Putumayo,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,0 km,-,-,-,-
192,Urubamba,"1,810 km","1,127 km",267 km,749 km,"1,921 km",743 km,750 km,"1,401 km",-,...,344 km,"1,496 km",427 km,"1,103 km","1,333 km",-,0 km,"2,133 km",86.3 km,"1,742 km"
193,Sullana,324 km,"1,217 km","2,099 km","1,518 km",716 km,"1,797 km","1,412 km",914 km,-,...,"2,021 km",996 km,"2,308 km","1,030 km",957 km,-,"2,124 km",0 km,"2,163 km",541 km
194,Quispicanchi,"1,849 km","1,166 km",180 km,788 km,"1,960 km",837 km,789 km,"1,440 km",-,...,258 km,"1,535 km",341 km,"1,142 km","1,372 km",-,86.1 km,"2,172 km",0 km,"1,781 km"


In [152]:
# prov_distances.to_excel('prov_distances.xlsx', index=False)
# prov_times.to_excel('prov_times.xlsx', index=False)

### Aplicando la función a las Matrices de Distritos

In [136]:
primera_fila = 1
primera_col = 1
ult_fila_mas_uno = 2
ult_col = len(distritos)+1

matrix_blocks(primera_fila, primera_col, ult_fila_mas_uno, ult_col, dist_distances, dist_times, origen_destino_dist)

1


In [140]:
dist_distances.head()

,Origin Province\Destination Province,Pariahuanca,Sondorillo,Tocache,Tintay Puncu,Curimana,Chazuta,Candarave,Marcara,Coyllurqui,...,Cochorco,Acos Vinchos,Colcamar,Pillpinto,Oropesa,Yauyos,La Banda de Shilcayo,Chirinos,Putumayo,Velille
0,Pariahuanca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sondorillo,"1,382 km",0 km,918 km,-,"1,289 km",657 km,"2,324 km",778 km,"2,089 km",...,790 km,"1,641 km",354 km,"2,248 km","2,173 km","1,336 km",609 km,146 km,-,"2,129 km"
2,Tocache,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Tintay Puncu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Curimana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
